<span style="font-size:25px">The central limit theorem.</span>


In [2]:
#We wish to illustrate the central limit theorem, which claims:

In [4]:
%%latex
$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\Var}{\mathrm{Var}}$
Sei $(X_i)_{i\in \mathbb{N}}$ eine Folge unabhängig, identisch verteilter Zufallsvariablen mit $\mbox{Var}(X_1)<\infty$. Dann gilt
\begin{equation}
\frac{\sum_{i=1}^nX_i-n\mathbb{E}[X_1]}{\sqrt{n\mbox{Var}(X_1)}}\to \mathcal{N}(0, 1)
\end{equation}
in Verteilung, also  für alle $z\in \mathbb{R}$
\begin{equation}
F_n(z)=\mathbb{P}\left( \frac{S_n-n\mathbb{E}[X_1]}{\sqrt{n\Var(X_1)}}\leq z\right)\to \Phi(z)
\end{equation}
wobei $\Phi$ die Verteilungsfunktion der Standardnormalverteilung darstellt. Wir machen für die Simulation folgende Annahmen:
1) $X_1\simeq Ber(p)$, 2) $z\in Z_N=\{4\frac{k}{N}, k=-N, ..., 0, ..., N\}$. Wir nennen im folgenden $N$ auch "resolution", also Auflösung.

<IPython.core.display.Latex object>

In [6]:
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import seaborn as sns
import numpy as np
import scipy.stats as stats
from scipy.stats import norm
from ipywidgets import interact
import ipywidgets as widgets
from scipy.special import comb
from scipy.stats import binom
"""
Die folgende Funktion nimmt an als Parameter:
p = Erfolgswahrscheinlichkeit
n = log_10(Wie viele Zufallsvariablen wir summieren)
resolution = Für wie viele Werte von z wir uns die Verteilungsfunktion ansehen möchten
"""
def Simulationslauf(p=0.5, n=1,  resolution=3):
    #Punkte, für die wir die Verteilungsfunktion auswerten möchten
    z=np.linspace(-4, 4, resolution)
    n=int(10**n)
    #updated z
    z_transformed=np.sqrt(n*p*(1-p))*z+n*p
    F_n=binom.cdf(np.round(z_transformed).astype(int), n=n, p=p)
    #print(prob)
    """
    #Calculate probabilities explicitly
    prob=np.zeros(len(z))
    for i in range(len(z_transformed)):
        # For each z[i], compute the cumulative probability (binomial CDF)
        prob[i] = np.sum([comb(n, k) * p**k * (1 - p)**(n - k) for k in range(int(z_transformed[i]) + 1)])

    However, this code is very slow, so instead we use the binom function from scipy.stats
    """
    # Use vectorized comparison to calculate the CDF values
    cdf_diff=np.diff(F_n)
    cdf_diff_norm=np.diff(norm.cdf(z))
    z_1=z[1:]
    #Plot results
    fig, axes=plt.subplots(nrows=1, ncols=2, figsize=(10, 6))
    fig.tight_layout()
    axes[0].step(z, F_n)
    axes[0].step(z, norm.cdf(z))
    axes[1].step(z_1, cdf_diff/(8/resolution))
    axes[1].step(z_1, cdf_diff_norm/(8/resolution))
    plt.show()
    delta=np.max(abs(F_n-norm.cdf(z)))
    print(f'Der Maximale Abstand zwischen Normalverteilung und empirischer Verteilung beträgt {delta}')
   
slider_n=widgets.FloatSlider(value=1.3, min=1.1, max=10, description='log_10(n)', description_width='auto', orientation='horizontal', layout=widgets.Layout(width='500px'), readout=True)
slider_res=widgets.IntSlider(value=10, min=3, max=200, description='Auflösung', orientation='horizontal', layout=widgets.Layout(width='500px'), readout=True)
slider_p=widgets.FloatSlider(value=0.5, min=0.01, max=0.99, description='p', orientation='horizontal', layout=widgets.Layout(width='500px'), readout=True)

interact(Simulationslauf, p=slider_p, n=slider_n, resolution=slider_res)

interactive(children=(FloatSlider(value=0.5, description='p', layout=Layout(width='500px'), max=0.99, min=0.01…

<function __main__.Simulationslauf(p=0.5, n=1, resolution=3)>

In [8]:
%%latex
Can we infer a speed of convergence in $n$ ? I.e. how fast does the quantity
\[
  err(n):=\sup_{z\in \mathbb{R}}|F_n(z)-\Phi(z)|
\]
converge to zero? We guess a power law decay, i.e. for some $\alpha\in \mathbb{R}^+$ make the Ansatz that $err(n)=O(n^{-\alpha})$. To infer $\alpha$ from the simulation, we use a doubly-logarithmic plot.

<IPython.core.display.Latex object>

In [10]:
p=0.4
resolution=2000
def Error_calculation(n):
    #Punkte, für die wir die Verteilungsfunktion auswerten möchten
    z=np.linspace(-4, 4, resolution)
    n=int(10**n)
    #updated z
    z_transformed=np.sqrt(n*p*(1-p))*z+n*p
    prob=binom.cdf(np.round(z_transformed).astype(int), n=n, p=p)
    return np.max(abs(prob-norm.cdf(z)))

def plotting(n=4):
    n_val=np.linspace(1, n, 100)
    Error_calc_vec=np.vectorize(Error_calculation)
    errors=Error_calc_vec(n_val)
    log_errors=np.log10(errors)
    slope, intercept = np.polyfit(n_val, log_errors,  1)
    fit=intercept+slope*n_val
    print(f'Slope is {slope}')
    plt.title('Doppelt logarithmischer Plot |F_n(z)-Phi(z)| vs. n')
    plt.xlabel('log_10(n)')
    plt.ylabel('log_10(|F_n(z)-Phi(z)|)')
    plt.plot(n_val, fit)
    plt.plot(n_val, log_errors)
    plt.show()

slider_n1=widgets.FloatSlider(value=1.3, min=1.1, max=10, description='log_10(n)', description_width='auto', orientation='horizontal', layout=widgets.Layout(width='500px'), readout=True)
interact(plotting, n=slider_n1)

interactive(children=(FloatSlider(value=1.3, description='log_10(n)', layout=Layout(width='500px'), max=10.0, …

<function __main__.plotting(n=4)>

In [12]:
%%latex
The above suggests that $\alpha=1/2$, i.e.
\[
\sup_{z\in \mathbb{R}}|F_n(z)-\Phi(z)|=O(n^{-1/2})
\]
This can indeed be proven (refer to the Berry-Esseen theorem).

<IPython.core.display.Latex object>